<a href="https://colab.research.google.com/github/goelnikhils-lgtm/languagemodels/blob/main/Python%20Async%20Usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#sample notebook for using async.io
#https://medium.com/@Shrishml/all-about-python-asyncio-ca1f5a8974b0
#https://krython.com/tutorial/python/asyncio-patterns-gather-and-wait/

In [9]:
import asyncio
import time
import random

#store all events in a time line for later analysis
async_timeline_events = []
#keep track of background events that we will gather at the end
tasks = []

#capture the start time to calculate relative timestamps
inital_timestamp = time.monotonic()

def log_event(timeline , task, event):
    """ Log an event with timestamp to visualize when operations start and end
        Args :
        timeline : list to append event to
        task: Name/ID of task
        event: Name of event

    """
    timestamp = time.monotonic() - inital_timestamp # calculate the start time
    timeline.append({
            "task":task,
            "event":event,
            "timestamp":timestamp
            })
    print(f"[{timestamp:.2f}] {event}: {task}")

async def async_db_operation(step):
    """ Simulate a database write operation like (saving analytics )
        In real time applications this would be writing to database which is I/O bound etc

    """
    task_id = f"log_{step}"
    log_event(async_timeline_events,task_id,"START")
    await asyncio.sleep(random.uniform(0.3,3.0))
    log_event(async_timeline_events,task_id,"END")

async def async_llm_call(prompt):
    """ Simulate a llm write operation like (saving analytics )
        In real time applications this would be writing to database which is I/O bound etc

    """
    task_id = f"log_{prompt}"
    log_event(async_timeline_events,task_id,"START")
    await asyncio.sleep(random.uniform(0.5,3.2))
    log_event(async_timeline_events,task_id,"END")
    return f"LLM Result:{prompt}"

async def async_http_call(endpoint):
    """ Simulate a HTTP call
        In real time applications this would be writing to database which is I/O bound etc

    """
    task_id = f"http_{endpoint.replace('/','')}"
    log_event(async_timeline_events,task_id,"START")
    await asyncio.sleep(random.uniform(0.4,4.0))
    log_event(async_timeline_events,task_id,"END")
    return f"HTTP Result from :{endpoint}"

async def run_async_pipeline(): #run the tasks in order all tasks will run asynchronously and time associated with http call 0.4 , 4.0 will the highst time for awaiting
    """ Main work flow that co-ordinates
        In real time applications this would be writing to database which is I/O bound etc

    """
    log_event(async_timeline_events,"main","START")

    #dependent task : we need the llm result before proceeding
    llm_result = await async_llm_call("What is the user intent")
    #background task: log the result to the database , but don't wait for it
    tasks.append(asyncio.create_task(async_db_operation("llm_result")))

    #dependent task : we need the http  result before proceeding
    http1 = await async_http_call("/api/data")
    #background task: log the API but don't block
    tasks.append(asyncio.create_task(async_db_operation("http1")))

    #dependent task : we need this data too before proceeding
    http2 = await async_http_call("/api/details")
    #background task: log the API but don't block
    tasks.append(asyncio.create_task(async_db_operation("http2")))

    #dependent task: generate summary from using the LLM with the collected data
    summary = await async_llm_call("Summarize Everything")
    #background task: log the API but don't block
    tasks.append(asyncio.create_task(async_db_operation("summary")))

    log_event(async_timeline_events,"main_pipeline","END")

    #wait for all background tasks to complete before exiting
    #his ensures all logging operations finish properly
    #this will run without gathering all tasks as well
    await asyncio.gather(*tasks) #gather all tasks when done

#fix for running asyncio in environments like Jupyter notebook
import nest_asyncio
nest_asyncio.apply()

#reset the timer before starting
intial_timestamp = time.monotonic()

#run the pipeline
print("\n---- Running Async Pipeline ---- \n")
asyncio.run(run_async_pipeline())


---- Running Async Pipeline ---- 

[0.00] START: main
[0.00] START: log_What is the user intent
[1.90] END: log_What is the user intent
[1.90] START: http_apidata
[1.90] START: log_llm_result
[2.79] END: log_llm_result
[3.69] END: http_apidata
[3.69] START: http_apidetails
[3.69] START: log_http1
[5.08] END: log_http1
[6.06] END: http_apidetails
[6.06] START: log_Summarize Everything
[6.06] START: log_http2
[7.02] END: log_Summarize Everything
[7.02] END: main_pipeline
[7.02] START: log_summary
[8.34] END: log_summary
[8.35] END: log_http2
